## Training the Convolutional Neural Network
This notebooks includes code to initialize and train the CNN. Output from this notebook includes: 
* Training history
* Trained weights of the CNN
* Class predictions for each day 

In [2]:
import numpy as np
import tensorflow as tf
import random

#### Set up reproducible conditions 
Set seed to generate reproducible results. 

**NOTE 1**: PYTHONHASHSEED=0 specified in kernel.json file (envs/share/jupyter/kernels/python3/kernel.json) (see https://stackoverflow.com/questions/58067359/is-there-a-way-to-set-pythonhashseed-for-a-jupyter-notebook-session/61953451#61953451). This variable must be set before kernel is launched. 

**NOTE 2**: current results are based on training on one CPU. Training on multiple CPUs will give very slightly different trained weights (but almost identical). 


In [ ]:
np.random.seed(101)
random.seed(201)
tf.random.set_seed(333)

In [ ]:
session_conf = tf.compat.v1.ConfigProto(device_count={'CPU': 1})
sess = tf.compat.v1.Session(config=session_conf)

In [ ]:
import importlib
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [ ]:
## project_utils are functions/variables defined by me
from project_utils import read_utils as read
from project_utils import utils as util
from project_utils import parameters as param
from project_utils import model_utils as mu

importlib.reload(read)
importlib.reload(util)
importlib.reload(mu)
importlib.reload(param)

#### load data and train model

In [ ]:
x_dat = read.get_hgt_slp_input()

In [ ]:
y_dat = util.get_precip_classes(pd.read_csv("../processed_data/region_mean_precip.csv")["prcp"], 
                               q = [0.95])

In [ ]:
y_dat_onehot = util.onehot(y_dat)

In [ ]:
ind = np.arange(len(y_dat))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test, ind_train, ind_test = train_test_split(x_dat, y_dat_onehot, ind, 
                                                                         test_size=0.25, random_state=42, 
                                                                         shuffle = True, stratify = y_dat)

In [ ]:
class_weights = util.class_weights(y_train[:,1].astype('int'))
class_weights

In [ ]:
model = mu.build_model(activity_reg = 0.001, dropout_rate = 0.2)

In [ ]:
model.load_weights("../processed_data/initial_weights.h5")

In [ ]:
model.summary()

In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, 
                                           restore_best_weights = True)

In [18]:
history = model.fit(x_train, y_train, 
          batch_size = 2048, 
          epochs = 400, 
          class_weight = class_weights, 
          validation_data = (x_test, y_test),          
          callbacks = [callback])

Epoch 1/400
6/6 [==============================] - 7s 950ms/step - loss: 11.6749 - accuracy: 0.3572 - precision: 0.0578 - recall: 0.7883 - auc: 0.5754 - val_loss: 10.6229 - val_accuracy: 0.4973 - val_precision: 0.0722 - val_recall: 0.7640 - val_auc: 0.6761
Epoch 2/400
6/6 [==============================] - 4s 707ms/step - loss: 10.3949 - accuracy: 0.5167 - precision: 0.0745 - recall: 0.7468 - auc: 0.6686 - val_loss: 9.4334 - val_accuracy: 0.5922 - val_precision: 0.0932 - val_recall: 0.8202 - val_auc: 0.7609
Epoch 3/400
6/6 [==============================] - 4s 695ms/step - loss: 9.2033 - accuracy: 0.5807 - precision: 0.0823 - recall: 0.7525 - auc: 0.7222 - val_loss: 8.3829 - val_accuracy: 0.6546 - val_precision: 0.1109 - val_recall: 0.8427 - val_auc: 0.8076
Epoch 4/400
6/6 [==============================] - 4s 687ms/step - loss: 8.1829 - accuracy: 0.6498 - precision: 0.1029 - recall: 0.7926 - auc: 0.7880 - val_loss: 7.4682 - val_accuracy: 0.6903 - val_precision: 0.1231 - val_recall: 0.

### model.evaluate(x_dat, y_dat_onehot)

In [20]:
model.save_weights("../processed_data/trained_weights.h5")

### Save history

In [21]:
hist_df = pd.DataFrame(history.history) 

In [22]:
hist_csv_file = '../processed_data/training_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

### Save predictions

In [23]:
class_predictions = model.predict(x_dat)

In [24]:
predict_df = pd.DataFrame(class_predictions)
predict_df = predict_df.rename(columns = {0: 'prob_0', 1: 'prob_1'})
predict_df['predicted_class'] = np.argmax(class_predictions, axis = 1)
predict_df['set'] = 'train'
predict_df.loc[ind_test, 'set'] = 'test'
predict_df['date'] = param.dates.values
predict_df['true_y'] = y_dat

In [25]:
predict_df.to_csv("../processed_data/predicted_class_data.csv", index = False)

### Model with geopotential height only

In [26]:
model_hgt = mu.build_model(activity_reg = 0.001, dropout_rate = 0.2, 
                                  input_channels = 1)

In [28]:
model_hgt.load_weights("../processed_data/initial_weights_hgt_only.h5")

In [29]:
x_dat_hgt = read.get_hgt_input()
x_train_hgt = x_dat_hgt[ind_train]
x_test_hgt = x_dat_hgt[ind_test]

In [30]:
history_hgt = model_hgt.fit(x_train_hgt, y_train, 
          batch_size = 2048, 
          epochs = 400, 
          class_weight = class_weights, 
          validation_data = (x_test_hgt, y_test),          
          callbacks = [callback])

Epoch 1/400
6/6 [==============================] - 5s 706ms/step - loss: 11.2326 - accuracy: 0.6829 - precision: 0.1208 - recall: 0.8104 - auc: 0.8620 - val_loss: 10.4668 - val_accuracy: 0.2550 - val_precision: 0.0435 - val_recall: 0.6629 - val_auc: 0.4528
Epoch 2/400
6/6 [==============================] - 4s 637ms/step - loss: 10.3435 - accuracy: 0.3906 - precision: 0.0529 - recall: 0.6522 - auc: 0.5374 - val_loss: 9.6618 - val_accuracy: 0.3684 - val_precision: 0.0595 - val_recall: 0.7865 - val_auc: 0.6261
Epoch 3/400
6/6 [==============================] - 5s 919ms/step - loss: 9.5297 - accuracy: 0.4618 - precision: 0.0649 - recall: 0.7537 - auc: 0.6528 - val_loss: 8.9248 - val_accuracy: 0.4771 - val_precision: 0.0735 - val_recall: 0.8146 - val_auc: 0.7281
Epoch 4/400
6/6 [==============================] - 5s 806ms/step - loss: 8.8066 - accuracy: 0.5312 - precision: 0.0766 - recall: 0.7712 - auc: 0.7167 - val_loss: 8.2498 - val_accuracy: 0.5588 - val_precision: 0.0886 - val_recall: 0.

In [31]:
model_hgt.evaluate(x_dat_hgt, y_dat_onehot)

446/446 [==============================] - 2s 4ms/step - loss: 0.5063 - accuracy: 0.8376 - precision: 0.2133 - recall: 0.8345 - auc: 0.9224


[0.5062995553016663,
 0.8376158475875854,
 0.2132616490125656,
 0.834502100944519,
 0.922390341758728]

In [32]:
model_hgt.save_weights("../processed_data/trained_weights_hgt_only.h5")

In [33]:
hist_df_hgt = pd.DataFrame(history_hgt.history) 

In [34]:
hist_csv_file = '../processed_data/training_history_hgt_only.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df_hgt.to_csv(f)

In [35]:
class_predictions_hgt = model_hgt.predict(x_dat_hgt)

In [36]:
predict_df_hgt = pd.DataFrame(class_predictions_hgt)
predict_df_hgt = predict_df_hgt.rename(columns = {0: 'prob_0', 1: 'prob_1'})
predict_df_hgt['predicted_class'] = np.argmax(class_predictions, axis = 1)
predict_df_hgt['set'] = 'train'
predict_df_hgt.loc[ind_test, 'set'] = 'test'
predict_df_hgt['date'] = param.dates.values
predict_df_hgt['true_y'] = y_dat

In [37]:
predict_df_hgt.to_csv("../processed_data/predicted_class_data_hgt_only.csv", index = False)